In [ ]:
import pandas as pd

In [ ]:
# Step 1: Read the file and parse it
from collections import defaultdict
import re

# Replace with your actual file path
file_path = 'outputs/txts/staffmeeting/staffmeeting_transcribed.txt'

with open(file_path, 'r', encoding='utf-8') as f:
    raw_lines = f.readlines()

# Extract all spoken text
all_texts = []

for line in raw_lines:
    match = re.search(r'Text:\s*(.*)', line)
    if match:
        all_texts.append(match.group(1).strip())

full_meeting_transcript = ' '.join(all_texts)

# Step 2: Extract speaker segments and group by speaker
speaker_texts = defaultdict(list)

for line in raw_lines:
    match = re.search(r'Speaker:\s*(SPEAKER_\d+).*?Text:\s*(.*)', line)
    if match:
        speaker = match.group(1)
        text = match.group(2).strip()
        speaker_texts[speaker].append(text)

# Step 3: Combine each speaker's full speech into one string
combined_speeches = {
    speaker: ' '.join(texts)
    for speaker, texts in speaker_texts.items()
}

# # remove 'SPEAKER_5' because they only have one line
# del combined_speeches['SPEAKER_05']

combined_speeches

In [ ]:
# Step 4: Define function to query your local Ollama LLM (DeepSeek-R1:14B)
import subprocess
import json

def query_ollama(prompt, model="deepseek-r1:14b"):
    command = ['ollama', 'run', model]
    process = subprocess.Popen(command, stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=prompt)
    return stdout.strip()

In [ ]:
# Step 5: Generate summary for each speaker
summaries = {}

for speaker, speech in combined_speeches.items():
    prompt = f"Summarise the key ideas expressed by {speaker} in the following transcript:\n\n{speech}\n\nKeep it concise and focus on the main points."
    print(f"Summarising {speaker}...")
    summary = query_ollama(prompt)
    summaries[speaker] = summary

In [ ]:
summaries = pd.DataFrame.from_dict(summaries, orient='index', columns=['Summary']).reset_index()
summaries.rename(columns={'index': 'Speaker'}, inplace=True)

In [ ]:
# divide the 'Summary' column into two columns: 'Summary' and 'Speaker' divided by "</think>"
summaries['thinking'] = summaries['Summary'].apply(lambda x: x.split('</think>')[0].strip())
summaries['Summary'] = summaries['Summary'].apply(lambda x: x.split('</think>')[1].strip() if '</think>' in x else x)

In [ ]:
summaries['thinking'] = summaries['thinking'].apply(lambda x: x.replace('<think>\n', ''))

In [ ]:
summaries.to_csv('outputs/summaries_30_july.csv', index=False)

In [ ]:
# Step 3: Create prompt and run the summary
prompt = (
    "You are a helpful assistant. Summarise the following meeting transcript into a clear, concise summary. "
    "Focus on the main ideas discussed, decisions made, and any concerns or tensions raised.\n\n"
    f"{full_meeting_transcript}"
)

summary = query_ollama(prompt)

In [ ]:
summary_text = summary.split('</think>')[1].strip()

print(summary_text)

Get the 

In [ ]:
summaries = pd.read_csv('outputs/summaries_30_july.csv')

In [ ]:
summaries